# Model Training - Hybrid GARCH-ML Model

This notebook demonstrates the training process for our hybrid volatility prediction model.

In [ ]:
import sys
sys.path.append('../src')

from models.garch_model import GARCHModel
from models.ml_model import MLModel
from models.hybrid_model import HybridModel
import yfinance as yf
import pandas as pd
import numpy as np

## 1. Data Preparation

In [ ]:
# Download data
ticker = "AAPL"
data = yf.download(ticker, start="2020-01-01")

# Calculate returns
returns = data['Adj Close'].pct_change().dropna()

# Split data
train_size = int(len(returns) * 0.8)
train_data = returns[:train_size]
test_data = returns[train_size:]

## 2. Train Individual Models

In [ ]:
# Train GARCH model
garch_model = GARCHModel()
garch_model.fit(train_data)

# Train ML model
ml_model = MLModel()
ml_model.fit(train_data)

## 3. Train Hybrid Model

In [ ]:
# Initialize and train hybrid model
hybrid_model = HybridModel(garch_model, ml_model)
hybrid_model.fit(train_data)

## 4. Model Evaluation

In [ ]:
# Make predictions
garch_pred = garch_model.predict(test_data)
ml_pred = ml_model.predict(test_data)
hybrid_pred = hybrid_model.predict(test_data)

# Calculate metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error

def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    return {'MSE': mse, 'MAE': mae}

# Print results
print("GARCH Metrics:", calculate_metrics(test_data, garch_pred))
print("ML Metrics:", calculate_metrics(test_data, ml_pred))
print("Hybrid Metrics:", calculate_metrics(test_data, hybrid_pred))